In [ ]:
!git clone https://github.com/mseaberg/lcls_beamline_toolbox
import os
os.chdir('lcls_beamline_toolbox')
!python3 -m pip install -e .
!pip install xraydb -q

In [ ]:
!pip install xopt==2.3.0 -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lcls_beamline_toolbox.xraywavetrace.beam1d as beam
import lcls_beamline_toolbox.xraywavetrace.optics1d as optics
import lcls_beamline_toolbox.xraywavetrace.beamline1d as beamline
import scipy.optimize as optimize
import copy
import scipy.spatial.transform as transform
from scipy.stats import qmc
from split_and_delay import SND

import torch
import gpytorch
import botorch

import warnings
warnings.filterwarnings("ignore")



from xopt import Xopt, Evaluator
from xopt.generators.bayesian import MOBOGenerator
from xopt.generators.bayesian import ExpectedImprovementGenerator, UpperConfidenceBoundGenerator
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs
from xopt import VOCS
from xopt import Xopt

import math
import pandas as pd

In [ ]:
def get_snd_outputs(inputs):
  """
  Study 1 Objective function. Takes an [n, 8] dim np array of
  [samples, ( t1_th1, t1_th2, chi1, chi2, t4_th1, t4_th2, chi1, chi2)].
  The entries lie in the uniform unit interval.
  They are scaled to lie in [-100e-6, +100e-6].
  Returns a torch tensor of dim [n, 2] of
  [samples, (do_sum_objective, IP_r_objective)]
  """
  inputs = inputs*200e-6 - 100e-6
  result = []

  for x in inputs:
    snd = SND(9500)
    x = np.array(x)

    snd.mvr_t1_th1(x[0])
    snd.mvr_t1_th2(x[1])
    snd.mvr_t1_chi1(x[2])
    snd.mvr_t1_chi2(x[3])
    snd.mvr_t4_th1(x[4])
    snd.mvr_t4_th2(x[5])
    snd.mvr_t4_chi1(x[6])
    snd.mvr_t4_chi2(x[7])

    snd.propagate_delay()

    dh1 = snd.get_t1_dh_sum()
    dd = snd.get_dd_sum()
    dh4 = snd.get_t4_dh_sum()
    do = snd.get_do_sum()
    my_IP_sum = snd.get_IP_sum()
    my_intensity = dh1 + dd + dh4 + do + my_IP_sum
    my_intensity *= (1.0 + np.random.rand() * 0.01)

    do_centroid = snd.get_IP_r()
    do_centroid *= (1.0 + np.random.rand() * 0.01)
    # do_centroid_x = snd.get_IP_cx()
    # do_centroid_y = snd.get_IP_cy()


    result.append( (np.log(do_centroid*1e6) - 2*np.log(my_intensity)).item())

  return torch.tensor(result, dtype=torch.float)

In [ ]:
def get_snd_outputs_detailed(inputs):
  """
  Takes the X opt features, returns the values of the Beam Position Error and
  Intensity at each sample.
  """
  inputs = inputs*200e-6 - 100e-6
  result = []

  for x in inputs:
    snd = SND(9500)
    x = np.array(x)

    snd.mvr_t1_th1(x[0])
    snd.mvr_t1_th2(x[1])
    snd.mvr_t1_chi1(x[2])
    snd.mvr_t1_chi2(x[3])
    snd.mvr_t4_th1(x[4])
    snd.mvr_t4_th2(x[5])
    snd.mvr_t4_chi1(x[6])
    snd.mvr_t4_chi2(x[7])

    snd.propagate_delay()

    dh1 = snd.get_t1_dh_sum()
    dd = snd.get_dd_sum()
    dh4 = snd.get_t4_dh_sum()
    do = snd.get_do_sum()
    my_IP_sum = snd.get_IP_sum()
    my_intensity = dh1 + dd + dh4 + do + my_IP_sum

    do_centroid = snd.get_IP_r()
    do_centroid_x = snd.get_IP_cx()
    do_centroid_y = snd.get_IP_cy()

    result.append([do_centroid*1e6, my_intensity])

  return torch.tensor(result, dtype=torch.float)

In [ ]:
def eval_function(input_dict: dict) -> dict:
  x1, x2, x3, x4, x5, x6, x7, x8 = input_dict["x1"], input_dict["x2"], input_dict["x3"], input_dict["x4"], input_dict["x5"], input_dict["x6"], input_dict["x7"], input_dict["x8"]
  Xinp = np.expand_dims(np.array([x1, x2, x3, x4, x5, x6, x7, x8]), axis=0)
  output = get_snd_outputs(Xinp).squeeze()
  f = output.item()
  return {"f": f}

In [ ]:
def run_turbo(eval_function=eval_function, init_samples=64, len_chain=150):
  """
  Runs TurBO chain on the eval function, with init_samples initial samples followed
  by len_chain samples.
  Returns the Xopt object.
  """
  low = 0.0
  high = 1.0
  n_init = 64
  vocs = VOCS(
      variables = {"x1": [low, high],
                  "x2": [low, high],
                  "x3": [low, high],
                  "x4": [low, high],
                  "x5": [low, high],
                  "x6": [low, high],
                  "x7": [low, high],
                  "x8": [low, high]
                  },
      objectives = {"f": "MINIMIZE"},
    )
  evaluator = Evaluator(function=eval_function)
  generator = ExpectedImprovementGenerator(
      vocs=vocs, turbo_controller="optimize"
  )
  X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)
  X.generator.turbo_controller.scale_factor = 1.5
  X.generator.turbo_controller.success_tolerance = 10
  X.generator.turbo_controller.failure_tolerance = 10
  X.generator.turbo_controller.length = 1.0

  sampler = qmc.LatinHypercube(d=8)
  xs = sampler.random(n=n_init)
  init_samples = pd.DataFrame({f'x{i+1}': xs[:,i] for i in range(xs.shape[1])})
  X.evaluate_data(init_samples)

  X.generator.train_model()
  X.generator.turbo_controller.update_state(X.generator.data)
  X.generator.turbo_controller.get_trust_region(X.generator.model)

  for i in range(len_chain):
    # if i % 10 == 0:
    #   print(f"Step: {i+1}")
    model = X.generator.train_model()
    trust_region = X.generator.turbo_controller.get_trust_region(generator.model)\
          .squeeze()
    scale_factor = X.generator.turbo_controller.length
    region_width = trust_region[1] - trust_region[0]
    best_value = X.generator.turbo_controller.best_value

    n_successes = X.generator.turbo_controller.success_counter
    n_failures = X.generator.turbo_controller.failure_counter

    acq = X.generator.get_acquisition(model)

    X.step()

  X.generator.turbo_controller

  return X

In [ ]:
def get_optimum_details(X, plot=True):
  """
  Takes Xopt object, finds the sample with the minimum and prints the settings
  of this sample, the beam position error and the inensity at this setting.
  """
  min_idx = np.argmin(X.generator.data["f"])
  min_val = X.generator.data["f"][min_idx]
  X_min = [X.generator.data["x1"][min_idx],
          X.generator.data["x2"][min_idx],
            X.generator.data["x3"][min_idx],
          X.generator.data["x4"][min_idx],
          X.generator.data["x5"][min_idx],
          X.generator.data["x6"][min_idx],
            X.generator.data["x7"][min_idx],
            X.generator.data["x8"][min_idx]
          ]
  inputs = np.array(X_min)
  inputs = inputs[np.newaxis,:]
  outs = get_snd_outputs_detailed(inputs)
  print("Optimum Inputs: ", X_min)
  print("BPE: ", outs[0][0].item())
  print("Intensity:", outs[0][1].item())

  if plot:
    y1 = X.generator.data["f"]
    y1_mins = np.minimum.accumulate(y1)

    idx = np.arange(len(y1_mins))
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 7))
    ax0.plot(idx, y1_mins,'k')
    ax0.grid()
    ax0.set_xlabel("Sample Number")
    ax0.set_ylabel("Objective")
    ax1.plot(idx[-50:], y1_mins[-50:],'k')
    ax1.grid()
    ax1.set_ylabel("Objective");
    plt.show()


In [ ]:
for _ in range(5):
  X = run_turbo(eval_function=eval_function, init_samples=64, len_chain=250)
  get_optimum_details(X)
  del X

In [ ]:
y1 = X.generator.data["f"]
y1_mins = np.minimum.accumulate(y1)


idx = np.arange(len(y1_mins))
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 7))
ax0.plot(idx, y1_mins,'k')
ax0.grid()
ax0.set_xlabel("Sample Number")
ax0.set_ylabel("Objective")

ax1.plot(idx[-50:], y1_mins[-50:],'k')
ax1.grid()
ax1.set_ylabel("Objective");

In [ ]:
def plot_optima_from_generator_data(X, plot=True):
  """
  Takes Xopt object, returns and plots the (noise-less) data for
  beam position error and intensity.
  Args:
    X: Xopt object
    plot: boolean to plot or not
  Returns:
    tuple of beam position error sample and intensity sample histories.
  """
  features = ["x1",  "x2", "x3", "x4", "x5",  "x6", "x7", "x8"]
  points = X.generator.data[features].to_numpy()
  outs = get_snd_outputs_detailed(points).numpy()
  bpe_vals,  intensity_vals =   outs[:,0], outs[:,1]
  bpe_optima = np.minimum.accumulate(bpe_vals)
  intensity_optima = np.maximum.accumulate(intensity_vals)

  if plot:
    fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(7,10))
    ax1.plot(bpe_optima, 'k')
    ax1.scatter(np.arange(len(bpe_optima)), bpe_vals)
    ax1.set_xlabel("Sample Number")
    ax1.set_ylabel("BPE (microns)")


    ax2.plot(intensity_optima, 'k')
    ax2.scatter(np.arange(len(intensity_optima)), intensity_vals)
    ax2.set_xlabel("Sample Number")
    ax2.set_ylabel("Intensity")
    plt.show()

  return bpe_vals,  intensity_vals

In [ ]:
temp = plot_optima_from_generator_data(X)

In [ ]:
np.min(temp[0]), np.max(temp[1])

In [ ]:
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e4]
# Notes: Seems to over-value the BPE minimum compared to the Intensity maximum.
# BPE:  0.6
# Intensity: 38713

# BPE:  0.37
# Intensity: 28000

# BPE:  0.95
# Intensity: 28893

# BPE:  0.56
# Intensity: 66816

# BPE:  0.78
# Intensity: 59000

# BPE:  0.84
# Intensity: 26000
#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e2]
#Seems to be over-valuing the BPE optimum at the cost of the intensity optimum
# BPE:  0.8
# Intensity: 28523

# BPE:  0.44
# Intensity: 38794

# BPE:  0.54
# Intensity: 15594

# BPE:  0.48
# Intensity: 29349

# BPE:  0.84
# Intensity: 15868

# BPE:  0.62
# Intensity: 66873

#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e6]

# BPE:  0.29
# Intensity: 62596

# BPE:  1.2
# Intensity: 31887

# BPE:  2.3
# Intensity: 61435

# BPE:  0.31
# Intensity: 40029

# BPE:  0.76
# Intensity: 48991

# BPE:  0.21
# Intensity: 29005


#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e10]

# BPE:  0.88
# Intensity: 28331

# BPE:  0.13
# Intensity: 14723

# BPE:  0.41
# Intensity: 17541

# BPE:  0.13
# Intensity: 27413

In [ ]:
# Intensity * 1
# Optimum Inputs:  [0.5064263351469553, 0.5054263188990498, 0.633381694360709, 0.6826654086298786, 0.3505339102010231, 0.3519533064161716, 0.41184731751105114, 0.2741884602146126]
# BPE:  0.5955268144607544
# Intensity: 28671.541015625
# Optimum Inputs:  [0.5056442531395993, 0.498957047182233, 0.27637638777340445, 0.4058998797547668, 0.46977703151750616, 0.4770483601514455, 0.7112620998126364, 0.6036183238670412]
# BPE:  0.8379337191581726
# Intensity: 57668.82421875
# Optimum Inputs:  [0.5120611152966434, 0.4683089045845141, 0.4824055857167691, 0.35796090150929194, 0.5018114316758147, 0.5458927222874569, 0.5679998302098593, 0.5898099342854869]
# BPE:  0.7419799566268921
# Intensity: 45802.51171875
# Optimum Inputs:  [0.48472615884209547, 0.5695141296169646, 0.1967570123351664, 0.699420633425186, 0.10118573232387224, 0.0173388084532586, 0.3720696821342162, 0.7286989296912089]
# BPE:  1.2563250064849854
# Intensity: 14921.341796875
# Optimum Inputs:  [0.5424478983460869, 0.6024821523412782, 0.175633416458393, 0.15510283348010237, 0.6807083988380184, 0.3991200760554404, 0.4418534301305804, 0.3073933213688447]
# BPE:  353.2081298828125
# Intensity: 42112.1484375


# Intensity squared
# Optimum Inputs:  [0.5020072210723082, 0.47155989045098534, 0.5069976184671406, 0.8114177295940853, 0.5172207580627631, 0.5460369077138989, 0.5051078659428181, 0.18215934827458763]
# BPE:  2.474250078201294
# Intensity: 63138.96875
# Optimum Inputs:  [0.551329208134654, 0.5468231448581526, 0.501968301458998, 0.3505219689852883, 0.5311894448532202, 0.5356936677279305, 0.34703603434961333, 0.7989878538480442]
# BPE:  0.32299768924713135
# Intensity: 50238.890625
# Optimum Inputs:  [0.5014172545085596, 0.4222417428199838, 0.5843003243077302, 0.4775278749323921, 0.4061772521835029, 0.48473657252360697, 0.5340080802853364, 0.40435206735221507]
# BPE:  0.2527773976325989
# Intensity: 16183.2802734375
# Optimum Inputs:  [0.5420031067467932, 0.5633925555373094, 0.811972102931379, 0.995224903907833, 0.5348311487736931, 0.7810572448518631, 0.44381695934056636, 0.8750429747269451]
# BPE:  353.2081298828125
# Intensity: 29523.75390625
# Optimum Inputs:  [0.5198424219473955, 0.5537313350338104, 0.3927560035876874, 0.1509372167196022, 0.4399934501324041, 0.4061888409678041, 0.6262753799851613, 0.8268113403526995]
# BPE:  0.3780748248100281
# Intensity: 47364.30859375

#Intensity**5
# Optimum Inputs:  [0.4888519294153849, 0.457989538386019, 0.8382741742314996, 0.5942588500776371, 0.9855307191165074, 0.0, 0.48891776550470734, 0.5018091738804709]
# BPE:  353.2081298828125
# Intensity: 28127.998046875
# Optimum Inputs:  [0.5458750240615149, 0.5598235348300786, 0.0, 0.15365585115457175, 0.05082360074687623, 0.9252127352354376, 1.0, 0.7644878004399154]
# BPE:  353.2081298828125
# Intensity: 29640.380859375
# Optimum Inputs:  [0.4981587581669821, 0.4820524630412809, 0.43956533946321535, 0.7215275027458938, 0.5205020666369564, 0.5351987938522175, 0.1852575422185968, 0.6546446444342546]
# BPE:  1.6148204803466797
# Intensity: 62717.49609375
# Optimum Inputs:  [0.49297724432184925, 0.5124088161198512, 0.6504952544893768, 0.5333225224756037, 0.4392904592990474, 0.41976299928333827, 0.5150912302863849, 0.2990496028317855]
# BPE:  1.2065986394882202
# Intensity: 45681.6484375
# Optimum Inputs:  [0.5276414661993981, 0.5552020807485002, 1.0, 0.11859917256928833, 0.48329533276341957, 0.4569227698308048, 0.23799503480926085, 0.6359448337106235]
# BPE:  3.561697244644165
# Intensity: 63492.65234375

In [ ]:
#Intensity**2, Noise, 314 samples.
#X.generator.turbo_controller.scale_factor = 1.25
  # X.generator.turbo_controller.success_tolerance = 10
  # X.generator.turbo_controller.failure_tolerance = 10
  # X.generator.turbo_controller.length = 0.6
# Optimum Inputs:  [0.5363829445704176, 0.5659037187386242, 0.5669300706987271, 0.7767474331190324, 0.10995965026654916, 0.08081375150658296, 0.6583768845029824, 0.0009671551350726513]
# BPE:  0.6998624801635742
# Intensity: 28625.25

# Optimum Inputs:  [0.44570878427749994, 0.3998918493911712, 0.5715351750305934, 0.609182192948664, 0.5157891530190972, 0.5623032017555702, 0.38954753078185256, 0.42970485693626354]
# BPE:  1.1297509670257568
# Intensity: 39881.08203125

# Optimum Inputs:  [0.513939819141375, 0.5267023110310084, 0.5010479491943107, 0.9917707053304233, 0.44735455343379243, 0.43407989584396856, 0.08399108012428366, 0.425432342446693]
# BPE:  0.7460193634033203
# Intensity: 53533.44921875

# Optimum Inputs:  [0.527423896163525, 0.5208622203818006, 0.21002918334018067, 0.2707301443195908, 0.48034376344331314, 0.487223580534145, 0.6863070063395587, 0.8301217244050567]
# BPE:  0.3781445622444153
# Intensity: 59882.40625

# Optimum Inputs:  [0.49345050922502787, 0.44024930122692646, 0.37094082166125625, 0.11036132995387635, 0.5595076968056569, 0.6135322104797328, 0.7999466147502853, 0.7144455138896767]
# BPE:  1.4061925411224365
# Intensity: 48704.453125


#Intensity**2, Noise, 314 samples.
# X.generator.turbo_controller.scale_factor = 1.5
#   X.generator.turbo_controller.success_tolerance = 10
#   X.generator.turbo_controller.failure_tolerance = 10
#   X.generator.turbo_controller.length = 1.0

# Optimum Inputs:  [0.509301183380085, 0.495451042934416, 0.5052398049927695, 0.23402272529131485, 0.712239792827786, 0.7270200277409234, 0.6375474990408625, 0.6215357492859048]
# BPE:  1.130184531211853
# Intensity: 27970.23828125

# Optimum Inputs:  [0.48753873507630874, 0.5049170668380474, 0.379860827797895, 0.8001523884160501, 0.7787170556568186, 0.7612079299060049, 0.6424625668750114, 0.17956415476646878]
# BPE:  0.6466690897941589
# Intensity: 26443.49609375

# Optimum Inputs:  [0.4850578777746958, 0.43059855785987855, 0.9628405524197523, 0.06427329932158485, 0.5764878233331155, 0.6309389157397963, 0.9715505862073327, 0.0002467612936045704]
# BPE:  0.5559214353561401
# Intensity: 45301.84765625

# Optimum Inputs:  [0.5138048422377065, 0.5123274744001717, 0.7829805410751155, 0.28169762937380843, 0.641095869258619, 0.6424397318111054, 0.34405172454629906, 0.5908482868043888]
# BPE:  0.3834937512874603
# Intensity: 29283.46875

# Optimum Inputs:  [0.48982231939854964, 0.4567300439442223, 0.48480414290590584, 0.5664975583110285, 0.5833662659497765, 0.6158524698073379, 0.5592428358156301, 0.3888871604326925]
# BPE:  0.668344259262085
# Intensity: 44334.39453125

In [ ]:
#Step 1. Track len_min and max for the TC
#Step 2. Instead of optimize setting, use the safety version of turbo.
#Step 3. Use the 'use_low_noise_prior' flag for the gp_constructor.